In [1]:
import os
import zipfile
import pyreadstat
import pandas as pd

# Get a list of variable names (from https://rpubs.com/simaan84/bhc)

In [2]:
# I used the variable name list provided by Majeed Simaan, since I could not find the list.  
# data = {
#     'Description': ["id", "date", "parent_id", "country", "name", "name_legal", 
#                     "fed_district_code", "org_type", "charter_type", "fed_regulator2", 
#                     "fed_regulator", "lei", "lei", "city", "state"],
#     'Code': ["RSSD9001", "RSSD9999", "RSSD9364", "RSSD9005", "RSSD9010", "RSSD9017", 
#              "RSSD9032", "RSSD9047", "RSSD9048", "RSSD9217", "PRIM_FED_REG", "ID.LEI", 
#              "ID_LEI", "RSSD9130", "RSSD9200"]
# }


# Revised data dictionary
data = {
    'Description': ["id", "date", "parent_id", "country", "name", "name_legal", 
                    "fed_district_code", "org_type", "charter_type", "fed_regulator2", 
                    "fed_regulator", "lei", "lei", "city", "state",
                    "interest bearing deposits of the bank (domestic)",
                    "non-interest bearing deposits of the bank (domestic)",
                    "total deposits (domestic)",
                    "INTEREST-BEARING DEPOSITS (BHC CONSOL. U.S. OFC)",
                    "NONINTEREST-BEARING DEPOSITS (BHC CONSOL. FOREIGN OFC.)"],
    'Code': ["RSSD9001", "RSSD9999", "RSSD9364", "RSSD9005", "RSSD9010", "RSSD9017", 
             "RSSD9032", "RSSD9047", "RSSD9048", "RSSD9217", "PRIM_FED_REG", "ID.LEI", 
             "ID_LEI", "RSSD9130", "RSSD9200",
             "RCON6636", "RCON6631", "RCON", "BHDM6636", "BHDM6631"]
}


# Convert dictionary to DataFrame
var_df = pd.DataFrame(data)



In [3]:
var_df

,Description,Code
0,id,RSSD9001
1,date,RSSD9999
2,parent_id,RSSD9364
3,country,RSSD9005
4,name,RSSD9010
5,name_legal,RSSD9017
6,fed_district_code,RSSD9032
7,org_type,RSSD9047
8,charter_type,RSSD9048
9,fed_regulator2,RSSD9217


# import Call Reports

One Year Data

In [4]:
# # Set the data directory path 
# data_dir = '/Users/twylazhang/Desktop/Econ_DS_Research/background/COM'


# # Get a list of names of zip files, since there are four zip files. 
# zip_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.zip')]

# # Unzip the files
# for zip_file in zip_files:
#     with zipfile.ZipFile(zip_file, 'r') as zip_ref:
#         zip_ref.extractall(data_dir)

# # List of names of all .xpt files after unzipping
# xpt_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.xpt')]


# # Select the variables of interest
# variables_of_interest = var_df['Code'].tolist()  

# # Since each file have many other columns of data, 
# # we only want variables of interest. So we read the .xpt files and filter by variables of interest. 
# filtered_data_list = []
# for file_path in xpt_files:
#     df, meta = pyreadstat.read_xport(file_path, usecols=variables_of_interest)
#     filtered_data_list.append(df)

# # Combine data from all four files into a single data frame
# call = pd.concat(filtered_data_list, ignore_index=True)


Last Quater Data

In [5]:
# Set the path to the XPT file
file_path = '/Users/twylazhang/Desktop/Econ_banks_Research/background/COM/call1712.xpt'

# Read the XPT file into a DataFrame but only load the specified variables of interest
variables_of_interest = var_df['Code'].tolist()
call, meta = pyreadstat.read_xport(file_path, usecols=variables_of_interest)

# Rename the columns using var_df by mapping 'Code' to 'Description'
var_name_to_label = pd.Series(var_df['Description'].values, index=var_df['Code']).to_dict()
call.rename(columns=var_name_to_label, inplace=True)

# Replace parent_id values that are zero with the corresponding id values
if 'parent_id' in call.columns and 'id' in call.columns:
    call.loc[call['parent_id'] == 0, 'parent_id'] = call.loc[call['parent_id'] == 0, 'id']

# Drop all rows with any missing values
call.dropna(inplace=True)

# Display the updated DataFrame
call

,date,id,charter_type,name_legal,name,org_type,fed_district_code,city,country,state,fed_regulator2,lei,parent_id
0,20171231.0,37.0,200.0,BANK OF HANCOCK COUNTY,BANK OF HANCOCK CTY,1.0,6.0,SPARTA,UNITED STATES,GA,FDIC,0,37.0
1,20171231.0,242.0,200.0,FIRST COMMUNITY BANK XENIA-FLORA,FIRST CMNTY BK XENIA FLORA,1.0,8.0,XENIA,UNITED STATES,IL,FRS,0,3088643.0
2,20171231.0,279.0,300.0,"MINEOLA COMMUNITY BANK, SSB",MINEOLA CMNTY BK SSB,6.0,11.0,MINEOLA,UNITED STATES,TX,FDIC,0,3619720.0
3,20171231.0,354.0,200.0,BISON STATE BANK,BISON ST BK,1.0,10.0,BISON,UNITED STATES,KS,FDIC,0,354.0
4,20171231.0,457.0,200.0,LOWRY STATE BANK,LOWRY ST BK,1.0,9.0,LOWRY,UNITED STATES,MN,FDIC,0,1127016.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5957,20171231.0,5086072.0,200.0,BLUE GATE BANK,BLUE GATE BK,1.0,12.0,COSTA MESA,UNITED STATES,CA,FDIC,0,5086072.0
5958,20171231.0,5087752.0,0.0,HUDSON BRANCH,ROYAL BK OF CANADA HUDSON BR,0.0,0.0,JERSEY CITY,UNITED STATES,NJ,OCC,0,5087752.0
5959,20171231.0,5087949.0,400.0,"MB FINANCIAL INTERNATIONAL, INC.",MB FNCL INTL,1.0,4.0,ROSEMONT,UNITED STATES,IL,FRS,0,1090987.0
5960,20171231.0,5113866.0,0.0,NEW YORK BRANCH,TAIWAN BUS BK NY BR,0.0,0.0,NEW YORK,UNITED STATES,NY,FRS,0,5113866.0


In [6]:
call.describe()

,date,id,charter_type,org_type,fed_district_code,parent_id
count,5962.0,5.962000e+03,5962.000000,5962.000000,5962.000000,5.962000e+03
mean,20171231.0,1.007098e+06,208.473667,1.299732,7.158839,1.973392e+06
std,0.0,1.056318e+06,54.788961,1.289459,3.200313,1.261324e+06
min,20171231.0,3.700000e+01,0.000000,0.000000,0.000000,3.700000e+01
25%,20171231.0,3.276308e+05,200.000000,1.000000,6.000000,1.084502e+06
50%,20171231.0,6.519055e+05,200.000000,1.000000,7.000000,1.249011e+06
75%,20171231.0,9.642268e+05,200.000000,1.000000,10.000000,2.997466e+06
max,20171231.0,5.143788e+06,400.000000,11.000000,12.000000,5.170805e+06


In [7]:
call.to_csv('call.csv', index=False) 

In [8]:
# call.columns

In [9]:
# # Check if all three RCON variables exist in the call report columns
# required_rcon_vars = ['RCON6636', 'RCON6631', 'RCON']

# # Load the XPT file to get all available columns
# call_report, meta = pyreadstat.read_xport(file_path, metadataonly=True)

# # Check which RCON variables are present in the call report
# available_rcon_vars = [var for var in required_rcon_vars if var in call_report.columns]

# # Display the results
# available_rcon_vars

In [10]:
# recon_vars = [var for var in call_report.columns if 'RCON' in var]
# recon_vars

# FR Y-9C Reports


 one_year data

In [11]:
# # Set the directory paths
# data_dir = '/Users/twylazhang/Desktop/Econ_DS_Research/background/HOLD'
# # List of names of all CSV files in the directory
# csv_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.csv')]
# # Read and combine all CSV files into one DataFrame
# FR_report = pd.concat([pd.read_csv(file, low_memory=False) for file in csv_files], ignore_index=True)

data for the last quarter 

In [12]:
# Set the path to the CSV file
csv_file_path = '/Users/twylazhang/Desktop/Econ_banks_Research/background/HOLD/bhcf1712.csv'


# Filter the variables of interest to include only those available in the CSV file
available_columns = pd.read_csv(csv_file_path, nrows=0).columns.tolist()
valid_columns = [col for col in variables_of_interest if col in available_columns]

# Read the CSV file using only the validated columns of interest
FR_report = pd.read_csv(csv_file_path, usecols=valid_columns)

# Rename the columns using var_df by mapping 'Code' to 'Description'
var_name_to_label = pd.Series(var_df['Description'].values, index=var_df['Code']).to_dict()
FR_report.rename(columns=var_name_to_label, inplace=True)

# Drop rows with placeholder '--------' in the 'id' column, if 'id' is part of the DataFrame
if 'id' in FR_report.columns:
    FR_report = FR_report[FR_report['id'] != '--------']

# Drop all rows with any missing values
FR_report.dropna(inplace=True)

# Display the DataFrame
FR_report


/var/folders/cj/742s04b178ld3v0xjpn8xzt80000gn/T/ipykernel_66246/1547677207.py:10: DtypeWarning: Columns (0,1,7,1998,1999,2311,2340,2371) have mixed types. Specify dtype option on import or set low_memory=False.
  FR_report = pd.read_csv(csv_file_path, usecols=valid_columns)


,id,date,fed_district_code,NONINTEREST-BEARING DEPOSITS (BHC CONSOL. FOREIGN OFC.),INTEREST-BEARING DEPOSITS (BHC CONSOL. U.S. OFC),charter_type,city,country,parent_id,lei,name_legal,name,org_type,fed_regulator,state
1,1020180,20171231,9,3010842,7112735,500,SAINT PAUL,UNITED STATES,0,549300TXP74T8NJZJW60,BREMER FINANCIAL CORPORATION,BREMER FNCL CORP,1,FRS,MN
9,1020902,20171231,10,5333437,11382415,500,OMAHA,UNITED STATES,0,0,"FIRST NATIONAL OF NEBRASKA, INC.",FIRST NAT OF NE,1,FRS,NE
14,1022764,20171231,12,527193,4429023,500,HONOLULU,UNITED STATES,0,549300W3YEAOZ4KGG849,CENTRAL PACIFIC FINANCIAL CORP.,CENTRAL PACIFIC FC,1,FRS,HI
21,1025309,20171231,12,4297801,9636475,500,HONOLULU,UNITED STATES,0,5493006Q8BQ8AD8M2U33,BANK OF HAWAII CORPORATION,BANK OF HI CORP,1,FRS,HI
22,1025541,20171231,12,2198403,2630086,500,SAN RAFAEL,UNITED STATES,0,0,WESTAMERICA BANCORPORATION,WESTAMERICA BC,1,FRS,CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4549,5069897,20171231,12,327308.0,776996.0,500,WALNUT CREEK,UNITED STATES,0,549300UOLS54Q5ONIG33,BAYCOM CORP,BAYCOM CORP,1,FRS,CA
4553,5079476,20171231,2,151479.0,972424.0,500,YORKTOWN HEIGHTS,UNITED STATES,0,0,PCSB FINANCIAL CORPORATION,PCSB FC,1,FRS,NY
4576,5117323,20171231,1,354098.0,635384.0,500,KENNEBUNK,UNITED STATES,0,0,"KENNEBUNK SAVINGS BANCORP, MHC",KENNEBUNK SVGS BC MHC,6,FRS,ME
4580,5119363,20171231,8,2022.0,15070.0,500,SAINT LOUIS,UNITED STATES,0,549300IYPEEU6O6OCU81,"PARAMOUNT FINANCIAL GROUP, LLC",PARAMOUNT FNCL GRP LLC,11,FRS,MO


In [13]:
FR_report.to_csv('FR_Report.csv', index=False) 

In [14]:
FR_report[FR_report['parent_id'] == "0"]

,id,date,fed_district_code,NONINTEREST-BEARING DEPOSITS (BHC CONSOL. FOREIGN OFC.),INTEREST-BEARING DEPOSITS (BHC CONSOL. U.S. OFC),charter_type,city,country,parent_id,lei,name_legal,name,org_type,fed_regulator,state
1,1020180,20171231,9,3010842,7112735,500,SAINT PAUL,UNITED STATES,0,549300TXP74T8NJZJW60,BREMER FINANCIAL CORPORATION,BREMER FNCL CORP,1,FRS,MN
9,1020902,20171231,10,5333437,11382415,500,OMAHA,UNITED STATES,0,0,"FIRST NATIONAL OF NEBRASKA, INC.",FIRST NAT OF NE,1,FRS,NE
14,1022764,20171231,12,527193,4429023,500,HONOLULU,UNITED STATES,0,549300W3YEAOZ4KGG849,CENTRAL PACIFIC FINANCIAL CORP.,CENTRAL PACIFIC FC,1,FRS,HI
21,1025309,20171231,12,4297801,9636475,500,HONOLULU,UNITED STATES,0,5493006Q8BQ8AD8M2U33,BANK OF HAWAII CORPORATION,BANK OF HI CORP,1,FRS,HI
22,1025541,20171231,12,2198403,2630086,500,SAN RAFAEL,UNITED STATES,0,0,WESTAMERICA BANCORPORATION,WESTAMERICA BC,1,FRS,CA
25,1026632,20171231,12,812000,168864000,500,SAN FRANCISCO,UNITED STATES,0,0,"CHARLES SCHWAB CORPORATION, THE",CHARLES SCHWAB CORP,1,FRS,CA
26,1026801,20171231,12,1493337,1941819,500,FREMONT,UNITED STATES,0,549300HJ9U2LJ44PTD84,FREMONT BANCORPORATION,FREMONT BC,1,FRS,CA
27,1027004,20171231,12,23868342,28752733,500,SALT LAKE CITY,UNITED STATES,0,YYQWUR1Z2BCX32HQS333,ZIONS BANCORPORATION,ZIONS BC,1,FRS,UT
31,1029222,20171231,12,3846436,2700417,500,ONTARIO,UNITED STATES,0,0,CVB FINANCIAL CORP.,CVB FC,1,FRS,CA
35,1029464,20171231,12,2356637,3091662,500,SPOKANE,UNITED STATES,0,0,W.T.B. FINANCIAL CORPORATION,WTB FC,1,FRS,WA


# 10085 parent ids are zeroes. 
# Assign the parent id number the same value of the id column, i.e.

In [15]:
# Replace 'parent_id' values of 0 with 'id' values from the same rows
FR_report.loc[FR_report['parent_id'] == "0", 'parent_id'] = FR_report.loc[FR_report['parent_id'] == "0", 'id']

# Drop all rows that contain any missing values
# FR_report.dropna(inplace=True)


# Now there are no rows of data where the parent_id is 0, meaning that we do not know which bank holding company the bank belongs to. 
FR_report[FR_report['parent_id'] == "0"]


,id,date,fed_district_code,NONINTEREST-BEARING DEPOSITS (BHC CONSOL. FOREIGN OFC.),INTEREST-BEARING DEPOSITS (BHC CONSOL. U.S. OFC),charter_type,city,country,parent_id,lei,name_legal,name,org_type,fed_regulator,state


# Merging the Databset

Steps to merge on parent_id of call and id of FR:
1. Renaming Columns: appends "_call" to the names of all columns in the call dataframe, except for the "date" column, which is left unchanged for merging purposes.

2. Creating Common Identifiers: It creates a new column common_id in both call dataframe and FR dataframe. In call df, parent_id_call is changed and set to common_id; in FR df, id is changed and set to common_id. This common identifier is used for merging the two datasets.

3. Merging Datasets: The datasets are merged on the common_id_call and date.

In [16]:
# Step 1
new_columns = []
for col in call.columns:
    if col != 'date':
        new_columns.append(col + '_call')
    else:
        new_columns.append(col)
call.columns = new_columns

In [17]:
call

,date,id_call,charter_type_call,name_legal_call,name_call,org_type_call,fed_district_code_call,city_call,country_call,state_call,fed_regulator2_call,lei_call,parent_id_call
0,20171231.0,37.0,200.0,BANK OF HANCOCK COUNTY,BANK OF HANCOCK CTY,1.0,6.0,SPARTA,UNITED STATES,GA,FDIC,0,37.0
1,20171231.0,242.0,200.0,FIRST COMMUNITY BANK XENIA-FLORA,FIRST CMNTY BK XENIA FLORA,1.0,8.0,XENIA,UNITED STATES,IL,FRS,0,3088643.0
2,20171231.0,279.0,300.0,"MINEOLA COMMUNITY BANK, SSB",MINEOLA CMNTY BK SSB,6.0,11.0,MINEOLA,UNITED STATES,TX,FDIC,0,3619720.0
3,20171231.0,354.0,200.0,BISON STATE BANK,BISON ST BK,1.0,10.0,BISON,UNITED STATES,KS,FDIC,0,354.0
4,20171231.0,457.0,200.0,LOWRY STATE BANK,LOWRY ST BK,1.0,9.0,LOWRY,UNITED STATES,MN,FDIC,0,1127016.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5957,20171231.0,5086072.0,200.0,BLUE GATE BANK,BLUE GATE BK,1.0,12.0,COSTA MESA,UNITED STATES,CA,FDIC,0,5086072.0
5958,20171231.0,5087752.0,0.0,HUDSON BRANCH,ROYAL BK OF CANADA HUDSON BR,0.0,0.0,JERSEY CITY,UNITED STATES,NJ,OCC,0,5087752.0
5959,20171231.0,5087949.0,400.0,"MB FINANCIAL INTERNATIONAL, INC.",MB FNCL INTL,1.0,4.0,ROSEMONT,UNITED STATES,IL,FRS,0,1090987.0
5960,20171231.0,5113866.0,0.0,NEW YORK BRANCH,TAIWAN BUS BK NY BR,0.0,0.0,NEW YORK,UNITED STATES,NY,FRS,0,5113866.0


In [18]:
# Step 2
call['common_id'] = call['parent_id_call']
FR_report['common_id'] = FR_report['id']

# Step 3: Merge the two datasets on 'common_id' and 'date'
# call_FR_combined = pd.merge(call, FR_report, on=['common_id', 'date'])
# the merge does not work, there are some issues

# Because the datatype is different, merging does not work


In [19]:
print(call['common_id'].dtype, FR_report['common_id'].dtype)
print(call['date'].dtype, FR_report['date'].dtype)


float64 object
float64 object


In [20]:
# Convert 'common_id' and 'date' to float in both DataFrames
call['common_id'] = call['common_id'].astype(float)
FR_report['common_id'] = FR_report['common_id'].astype(float)

call['date'] = call['date'].astype(float)
FR_report['date'] = FR_report['date'].astype(float)

# Now try merging again
call_FR_combined = pd.merge(call, FR_report, on=['common_id', 'date'])


In [21]:
call_FR_combined

,date,id_call,charter_type_call,name_legal_call,name_call,org_type_call,fed_district_code_call,city_call,country_call,state_call,...,charter_type,city,country,parent_id,lei,name_legal,name,org_type,fed_regulator,state
0,20171231.0,2376.0,300.0,ORITANI BANK,ORITANI BK,1.0,2.0,TOWNSHIP OF WASHINGTON,UNITED STATES,NJ,...,500,TOWNSHIP OF WASHINGTON,UNITED STATES,0,0,ORITANI FINANCIAL CORP,ORITANI FNCL CORP,1,FRS,NJ
1,20171231.0,3971.0,300.0,MIDCOUNTRY BANK,MIDCOUNTRY BK,1.0,9.0,BLOOMINGTON,UNITED STATES,MN,...,500,MACON,UNITED STATES,0,0,MIDCOUNTRY FINANCIAL CORP,MIDCOUNTRY FNCL CORP,1,FRS,GA
2,20171231.0,5210.0,200.0,ACNB BANK,ACNB BK,1.0,3.0,GETTYSBURG,UNITED STATES,PA,...,500,GETTYSBURG,UNITED STATES,0,0,ACNB CORPORATION,ACNB CORP,1,FRS,PA
3,20171231.0,7009.0,200.0,BESSEMER TRUST COMPANY,BESSEMER TC,1.0,2.0,WOODBRIDGE,UNITED STATES,NJ,...,500,WOODBRIDGE,UNITED STATES,0,0,"BESSEMER GROUP, INCORPORATED, THE",BESSEMER GRP,1,FRS,NJ
4,20171231.0,976703.0,200.0,"BESSEMER TRUST COMPANY, N.A.",BESSEMER TC NA,1.0,2.0,NEW YORK,UNITED STATES,NY,...,500,WOODBRIDGE,UNITED STATES,0,0,"BESSEMER GROUP, INCORPORATED, THE",BESSEMER GRP,1,FRS,NJ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
814,20171231.0,3918898.0,200.0,"CIT BANK, NATIONAL ASSOCIATION",CIT BK NA,1.0,12.0,PASADENA,UNITED STATES,CA,...,500,LIVINGSTON,UNITED STATES,1036967,N8O96ZZJQRFYQUJY7K79,CIT GROUP INC.,CIT GROUP,1,FRS,NJ
815,20171231.0,3938186.0,200.0,"BANKUNITED, NATIONAL ASSOCIATION",BANKUNITED NA,1.0,6.0,MIAMI LAKES,UNITED STATES,FL,...,500,MIAMI LAKES,UNITED STATES,0,96OVUWEW5D4YX6PFXD91,"BANKUNITED, INC.",BANKUNITED,1,FRS,FL
816,20171231.0,4041421.0,200.0,"FLORIDA COMMUNITY BANK, NATIONAL ASSOCIATION",FLORIDA CMNTY BK NA,1.0,6.0,WESTON,UNITED STATES,FL,...,500,WESTON,UNITED STATES,0,0,"FCB FINANCIAL HOLDINGS, INC.",FCB FNCL HOLDS,1,FRS,FL
817,20171231.0,4210227.0,200.0,NBH BANK,NBH BK,1.0,10.0,GREENWOOD VILLAGE,UNITED STATES,CO,...,500,GREENWOOD VILLAGE,UNITED STATES,0,0,NATIONAL BANK HOLDINGS CORPORATION,NATIONAL BK HOLDS CORP,1,FRS,CO


In [22]:
# Define the list of required variables
required_vars = [
    'id_call', 'name_legal_call', 'parent_id_call', 'common_id', 'id', 
    'NONINTEREST-BEARING DEPOSITS (BHC CONSOL. FOREIGN OFC.)',
    'INTEREST-BEARING DEPOSITS (BHC CONSOL. U.S. OFC)', 'parent_id', 'name_legal'
]

# Filter the DataFrame to keep only the required variables
filtered_call_FR_combined = call_FR_combined[required_vars]

# Display the filtered DataFrame
filtered_call_FR_combined.head(10)

,id_call,name_legal_call,parent_id_call,common_id,id,NONINTEREST-BEARING DEPOSITS (BHC CONSOL. FOREIGN OFC.),INTEREST-BEARING DEPOSITS (BHC CONSOL. U.S. OFC),parent_id,name_legal
0,2376.0,ORITANI BANK,2692892.0,2692892.0,2692892,163460.0,2810723.0,0,ORITANI FINANCIAL CORP
1,3971.0,MIDCOUNTRY BANK,3839902.0,3839902.0,3839902,100156.0,502011.0,0,MIDCOUNTRY FINANCIAL CORP
2,5210.0,ACNB BANK,1117464.0,1117464.0,1117464,54138.0,1244354.0,0,ACNB CORPORATION
3,7009.0,BESSEMER TRUST COMPANY,1246159.0,1246159.0,1246159,196425.0,2555880.0,0,"BESSEMER GROUP, INCORPORATED, THE"
4,976703.0,"BESSEMER TRUST COMPANY, N.A.",1246159.0,1246159.0,1246159,196425.0,2555880.0,0,"BESSEMER GROUP, INCORPORATED, THE"
5,2845782.0,"BESSEMER TRUST COMPANY OF CALIFORNIA, NATIONAL...",1246159.0,1246159.0,1246159,196425.0,2555880.0,0,"BESSEMER GROUP, INCORPORATED, THE"
6,3310287.0,"BESSEMER TRUST COMPANY OF DELAWARE, NATIONAL A...",1246159.0,1246159.0,1246159,196425.0,2555880.0,0,"BESSEMER GROUP, INCORPORATED, THE"
7,9807.0,AMBOY BANK,2611718.0,2611718.0,2611718,69525.0,1953369.0,0,AMBOY BANCORPORATION
8,12311.0,"HUNTINGTON NATIONAL BANK, THE",1068191.0,1068191.0,1068191,21680341.0,55497578.0,0,HUNTINGTON BANCSHARES INCORPORATED
9,12946.0,FARMERS BANK & TRUST COMPANY,1135413.0,1135413.0,1135413,157027.0,1060253.0,0,MAGNOLIA BANKING CORPORATION


In [23]:
filtered_call_FR_combined.head(10).to_csv('first10_call_FR_combined.csv', index=False)

In [24]:
call_FR_combined.to_csv('call_FR_combined.csv', index=False)

In [25]:
 # Compute the number of unique subsidiaries for each group
num_sub = call_FR_combined.groupby(['common_id', 'date']).agg(number_subs_1=('id_call', 'nunique')).reset_index()

# Print the results
print(num_sub)

     common_id        date  number_subs_1
0    1020180.0  20171231.0              2
1    1020902.0  20171231.0              1
2    1022764.0  20171231.0              1
3    1025309.0  20171231.0              1
4    1025541.0  20171231.0              1
..         ...         ...            ...
615  5069897.0  20171231.0              1
616  5079476.0  20171231.0              2
617  5117323.0  20171231.0              1
618  5119363.0  20171231.0              1
619  5124938.0  20171231.0              1

[620 rows x 3 columns]


In [26]:
num_sub.describe()

,common_id,date,number_subs_1
count,6.200000e+02,620.0,620.000000
mean,2.178799e+06,20171231.0,1.320968
std,1.146778e+06,0.0,1.168465
min,1.020180e+06,20171231.0,1.000000
25%,1.128181e+06,20171231.0,1.000000
50%,1.884500e+06,20171231.0,1.000000
75%,3.135300e+06,20171231.0,1.000000
max,5.124938e+06,20171231.0,17.000000


# from https://rpubs.com/simaan84/bhc
We observe that, mainly, there is a one-to-one relationship, with the exception for a number of cases. For instance, we note that in the last quarter of 2017, the Wintrust Financial Corporation had 17 subsidiaries:

In [27]:
# Step 1: Find IDs with 17 subsidiaries
id17 = num_sub.loc[num_sub['number_subs_1'] == 17, 'common_id'].unique()

# Step 2: Get the list of names for these IDs at the specific date
WT_list = call_FR_combined[(call_FR_combined['common_id'].isin(id17)) & (call_FR_combined['date'] == 20171231)]['name_call']

# Step 3: Print sorted list of names
print("\n".join(sorted(WT_list.astype(str))))


BARRINGTON B&TC NA
BEVERLY B&TC NA
CHICAGO TC NA
CRYSTAL LAKE B&TC NA
FIFC EDGE INTL CORP
HINSDALE B&TC
LAKE FOREST B&TC NA
LIBERTYVILLE B&TC
NORTHBROOK B&TC
OLD PLANK TRAIL CMNTY BK NA
SCHAUMBURG B&TC NA
ST CHARLES B&TC
STATE BK OF THE LAKES
TOWN BK
VILLAGE B&TC
WHEATON B&TC
WINTRUST BK


On the other hand, if we look at JP Morgan, for instance, we observe that it has two main subsidiaries at the end of 2017. This is in line with the 10-K report from the SEC (see link). Nevertheless, it is also appears that each subsidiary has a number of other subsidiaries in the annual report, which our data does not seem to convey such complexity in the structure:

In [28]:
# Filtering the DataFrame for JP Morgan entries at the end of 2017
JP_list = call_FR_combined[(call_FR_combined['common_id'] == 1039502) & (call_FR_combined['date'] == 20171231)]['name_legal_call']

# Sorting the names and printing each on a new line
sorted_JP_list = sorted(JP_list.astype(str)) 
print("\n".join(sorted_JP_list))

J.P. MORGAN INTERNATIONAL FINANCE LIMITED
JPMORGAN CHASE BANK, NATIONAL ASSOCIATION


# Final Data



In [29]:
# copy of the call_FR_combined
ds_final = call_FR_combined[['date', 'name_legal_call', 'lei_call', 'id_call', 'name_legal', 'lei', 'id']].copy()

# Compute the number of subsidiaries for each BHC in the sample
# Using transform to add the count of subsidiaries directly into the dataframe
ds_final['no_sub'] = ds_final.groupby(['id', 'date'])['id'].transform('count')

ds_final


,date,name_legal_call,lei_call,id_call,name_legal,lei,id,no_sub
0,20171231.0,ORITANI BANK,549300XRBLWUN6J76L65,2376.0,ORITANI FINANCIAL CORP,0,2692892,1
1,20171231.0,MIDCOUNTRY BANK,0,3971.0,MIDCOUNTRY FINANCIAL CORP,0,3839902,1
2,20171231.0,ACNB BANK,0,5210.0,ACNB CORPORATION,0,1117464,1
3,20171231.0,BESSEMER TRUST COMPANY,549300NEQHXUX4MZIH22,7009.0,"BESSEMER GROUP, INCORPORATED, THE",0,1246159,4
4,20171231.0,"BESSEMER TRUST COMPANY, N.A.",549300OTTCSOD5OCZ143,976703.0,"BESSEMER GROUP, INCORPORATED, THE",0,1246159,4
...,...,...,...,...,...,...,...,...
814,20171231.0,"CIT BANK, NATIONAL ASSOCIATION",FVPV0TNOIM6NUIE59530,3918898.0,CIT GROUP INC.,N8O96ZZJQRFYQUJY7K79,1036967,1
815,20171231.0,"BANKUNITED, NATIONAL ASSOCIATION",BFLFJ4PXRGDML87ZYT33,3938186.0,"BANKUNITED, INC.",96OVUWEW5D4YX6PFXD91,4028712,1
816,20171231.0,"FLORIDA COMMUNITY BANK, NATIONAL ASSOCIATION",549300SGSDB4HQJSLK07,4041421.0,"FCB FINANCIAL HOLDINGS, INC.",0,3944628,1
817,20171231.0,NBH BANK,549300T7AIY7RU1YDP31,4210227.0,NATIONAL BANK HOLDINGS CORPORATION,0,3973888,1


In [30]:
ds_final.to_csv('call_FR_combined_only_name_id.csv', index=False)

# Publically Traded Banks




In [31]:
# Load the dataset from the Federal Reserve Bank of New York that lists publicly traded banks
url = "https://www.newyorkfed.org/medialibrary/media/research/banking_research/data/crsp_20161231.csv"
crsp = pd.read_csv(url)

# Filter ds_final to include only IDs present in the CRSP dataset to find publicly traded banks
ds_final_public = ds_final[ds_final['id'].isin(crsp['entity'])]

# Count unique publicly traded companies and print the result
unique_public_companies = len(ds_final_public['id'].unique())
print(f"Total publicly traded companies: {unique_public_companies}")

Total publicly traded companies: 279


In [32]:
# Convert both 'id' in ds_final and 'entity' in crsp to integers
ds_final['id'] = pd.to_numeric(ds_final['id'], errors='coerce').astype('Int64')
crsp['entity'] = pd.to_numeric(crsp['entity'], errors='coerce').astype('Int64')

# Filter ds_final to include only IDs present in the crsp dataset
ds_final_public = ds_final[ds_final['id'].isin(crsp['entity'].dropna())]

# Count unique publicly traded companies and print the result
unique_public_companies = len(ds_final_public['id'].unique())
print(f"Total publicly traded companies: {unique_public_companies}")

Total publicly traded companies: 301


# Compare Deposits 

In [33]:
# Load data
call_report_path = 'path_to_call_report.csv'
fyr_data_path = 'path_to_fyr_data.csv'

call_report = pd.read_csv(call_report_path)
fyr_data = pd.read_csv(fyr_data_path)

# Keep specified variables
call_report = call_report[['ID_call', 'parent_call', 'RCON6636', 'RCON6631']]
fyr_data = fyr_data[['id_FYR', 'BHDM6636', 'BHDM6631']]

# Rename columns for clarity
call_report.columns = ['ID_call', 'parent_call', 'interest_deposits_call', 'non_interest_deposits_call']
fyr_data.columns = ['id_FYR', 'interest_deposits_FYR', 'non_interest_deposits_FYR']

# Merge data on parent_call and id_FYR
merged_data = call_report.merge(fyr_data, left_on='parent_call', right_on='id_FYR', how='left')

# Function to calculate and compare deposits
def compare_deposits(df):
    results = []
    for parent in df['parent_call'].unique():
        parent_data = df[df['parent_call'] == parent]
        if parent in parent_data['ID_call'].values:
            parent_row = parent_data[parent_data['ID_call'] == parent]
            ratio_interest = parent_row['interest_deposits_call'].sum() / parent_row['interest_deposits_FYR'].sum()
            ratio_non_interest = parent_row['non_interest_deposits_call'].sum() / parent_row['non_interest_deposits_FYR'].sum()
            results.append((parent, ratio_interest, ratio_non_interest))
        else:
            sum_interest = parent_data['interest_deposits_call'].sum()
            sum_non_interest = parent_data['non_interest_deposits_call'].sum()
            fyr_interest = parent_data['interest_deposits_FYR'].sum()
            fyr_non_interest = parent_data['non_interest_deposits_FYR'].sum()
            ratio_interest = sum_interest / fyr_interest
            ratio_non_interest = sum_non_interest / fyr_non_interest
            results.append((parent, ratio_interest, ratio_non_interest))
    return pd.DataFrame(results, columns=['parent_call', 'ratio_interest', 'ratio_non_interest'])

# Apply the function and display results
results = compare_deposits(merged_data)
results


FileNotFoundError: [Errno 2] No such file or directory: 'path_to_call_report.csv'